## 액터 작업 정의

In [1]:
from queue import Queue
from threading import Thread, Event

# Sentinel used for shutdown
class ActorExit(Exception):
    pass

class Actor:
    def __init__(self):
        self._mailbox = Queue()

    def send(self, msg):
        '''
        Send a message to the actor
        '''
        self._mailbox.put(msg)

    def recv(self):
        '''
        Receive an incoming message
        '''
        msg = self._mailbox.get()
        if msg is ActorExit:
            raise ActorExit()
        return msg

    def close(self):
        '''
        Close the actor, thus shutting it down
        '''
        self.send(ActorExit)

    def start(self):
        '''
        Start concurrent execution
        '''
        self._terminated = Event()
        t = Thread(target=self._bootstrap)
        t.daemon = True
        t.start()

    def _bootstrap(self):
        try:
            self.run()
        except ActorExit:
            pass
        finally:
            self._terminated.set()

    def join(self):
        self._terminated.wait()

    def run(self):
        '''
        Run method to be implemented by the user
        '''
        while True:
            msg = self.recv()

# Sample ActorTask
class PrintActor(Actor):
    def run(self):
        while True:
            msg = self.recv()
            print("Got:", msg)

if __name__ == '__main__':
    # Sample use
    p = PrintActor()
    p.start()
    p.send("Hello")
    p.send("World")
    p.close()
    p.join()


Got: Hello
Got: World


In [3]:
class TaggedActor(Actor):
    def run(self):
        while True:
             tag, *payload = self.recv()
             getattr(self,"do_"+tag)(*payload)
    
    # Methods correponding to different message tags
    def do_A(self, x):
        print("Running A", x)

    def do_B(self, x, y):
        print("Running B", x, y)

# Example
if __name__ == '__main__':
    a = TaggedActor()
    a.start()
    a.send(('A', 1))      # Invokes do_A(1)
    a.send(('B', 2, 3))   # Invokes do_B(2,3)
    a.close()
    a.join()

Running A 1
Running B 2 3


In [4]:
from threading import Event

class Result:
    def __init__(self):
        self._evt = Event()
        self._result = None

    def set_result(self, value):
        self._result = value
        self._evt.set()

    def result(self):
        self._evt.wait()
        return self._result

class Worker(Actor):
    def submit(self, func, *args, **kwargs):
        r = Result()
        self.send((func, args, kwargs, r))
        return r

    def run(self):
        while True:
            func, args, kwargs, r = self.recv()
            r.set_result(func(*args, **kwargs))

# Example use
if __name__ == '__main__':
    worker = Worker()
    worker.start()
    r = worker.submit(pow, 2, 3)
    print(r.result())
    worker.close()
    worker.join()

8


## 메시지 출판/구독 구현

In [5]:
from collections import defaultdict

class Exchange:
    def __init__(self):
        self._subscribers = set()

    def attach(self, task):
        self._subscribers.add(task)

    def detach(self, task):
        self._subscribers.remove(task)

    def send(self, msg):
        for subscriber in self._subscribers:
            subscriber.send(msg)

# Dictionary of all created exchanges
_exchanges = defaultdict(Exchange)

# Return the Exchange instance associated with a given name
def get_exchange(name):
    return _exchanges[name]

if __name__ == '__main__':
    # Example task (just for testing)
    class Task:
        def __init__(self, name):
            self.name = name
        def send(self, msg):
            print('{} got: {!r}'.format(self.name, msg))

    task_a = Task('A')
    task_b = Task('B')

    exc = get_exchange('spam')
    exc.attach(task_a)
    exc.attach(task_b)
    exc.send('msg1')
    exc.send('msg2')

    exc.detach(task_a)
    exc.detach(task_b)
    exc.send('msg3')

A got: 'msg1'
B got: 'msg1'
A got: 'msg2'
B got: 'msg2'


In [6]:
from contextlib import contextmanager
from collections import defaultdict

class Exchange:
    def __init__(self):
        self._subscribers = set()

    def attach(self, task):
        self._subscribers.add(task)

    def detach(self, task):
        self._subscribers.remove(task)

    @contextmanager
    def subscribe(self, *tasks):
        for task in tasks:
            self.attach(task)
        try:
            yield
        finally:
            for task in tasks:
                self.detach(task)

    def send(self, msg):
        for subscriber in self._subscribers:
            subscriber.send(msg)


# Dictionary of all created exchanges
_exchanges = defaultdict(Exchange)

# Return the Exchange instance associated with a given name
def get_exchange(name):
    return _exchanges[name]

# Example of using the subscribe() method
if __name__ == '__main__':
    # Example task (just for testing)
    class Task:
        def __init__(self, name):
            self.name = name
        def send(self, msg):
            print('{} got: {!r}'.format(self.name, msg))

    task_a = Task('A')
    task_b = Task('B')

    exc = get_exchange('spam')
    with exc.subscribe(task_a, task_b):
        exc.send('msg1')
        exc.send('msg2')

    exc.send('msg3')

A got: 'msg1'
B got: 'msg1'
A got: 'msg2'
B got: 'msg2'


## 스레드의 대안으로 제너레이터 사용

In [7]:
# A very simple example of a coroutine/generator scheduler

# Two simple generator functions
def countdown(n):
    while n > 0:
        print("T-minus", n)
        yield
        n -= 1
    print("Blastoff!")

def countup(n):
    x = 0
    while x < n:
        print("Counting up", x)
        yield
        x += 1

from collections import deque

class TaskScheduler:
    def __init__(self):
        self._task_queue = deque()

    def new_task(self, task):
        '''
        Admit a newly started task to the scheduler
        '''
        self._task_queue.append(task)

    def run(self):
        '''
        Run until there are no more tasks
        '''
        while self._task_queue:
            task = self._task_queue.popleft()
            try:
                # Run until the next yield statement
                next(task)
                self._task_queue.append(task)
            except StopIteration:
                # Generator is no longer executing
                pass

# Example use
sched = TaskScheduler()
sched.new_task(countdown(10))
sched.new_task(countdown(5))
sched.new_task(countup(15))
sched.run()

T-minus 10
T-minus 5
Counting up 0
T-minus 9
T-minus 4
Counting up 1
T-minus 8
T-minus 3
Counting up 2
T-minus 7
T-minus 2
Counting up 3
T-minus 6
T-minus 1
Counting up 4
T-minus 5
Blastoff!
Counting up 5
T-minus 4
Counting up 6
T-minus 3
Counting up 7
T-minus 2
Counting up 8
T-minus 1
Counting up 9
Blastoff!
Counting up 10
Counting up 11
Counting up 12
Counting up 13
Counting up 14


In [8]:
from collections import deque

class ActorScheduler:
    def __init__(self):
        self._actors = { }          # Mapping of names to actors
        self._msg_queue = deque()   # Message queue
    
    def new_actor(self, name, actor):
        '''
        Admit a newly started actor to the scheduler and give it a name
        '''
        self._msg_queue.append((actor,None))
        self._actors[name] = actor

    def send(self, name, msg):
        '''
        Send a message to a named actor
        '''
        actor = self._actors.get(name)
        if actor:
            self._msg_queue.append((actor,msg))

    def run(self):
        '''
        Run as long as there are pending messages.
        '''
        while self._msg_queue:
            actor, msg = self._msg_queue.popleft()
            try:
                 actor.send(msg)
            except StopIteration:
                 pass

# Example use
if __name__ == '__main__':
    def printer():
        while True:
            msg = yield
            print('Got:', msg)

    def counter(sched):
        while True:
            # Receive the current count
            n = yield    
            if n == 0:
                break
            # Send to the printer task
            sched.send('printer', n)
            # Send the next count to the counter task (recursive)
            sched.send('counter', n-1)

    sched = ActorScheduler()
    # Create the initial actors
    sched.new_actor('printer', printer())
    sched.new_actor('counter', counter(sched))

    # Send an initial message to the counter to initiate
    sched.send('counter', 10000)
    sched.run()

Got: 10000
Got: 9999
Got: 9998
Got: 9997
Got: 9996
Got: 9995
Got: 9994
Got: 9993
Got: 9992
Got: 9991
Got: 9990
Got: 9989
Got: 9988
Got: 9987
Got: 9986
Got: 9985
Got: 9984
Got: 9983
Got: 9982
Got: 9981
Got: 9980
Got: 9979
Got: 9978
Got: 9977
Got: 9976
Got: 9975
Got: 9974
Got: 9973
Got: 9972
Got: 9971
Got: 9970
Got: 9969
Got: 9968
Got: 9967
Got: 9966
Got: 9965
Got: 9964
Got: 9963
Got: 9962
Got: 9961
Got: 9960
Got: 9959
Got: 9958
Got: 9957
Got: 9956
Got: 9955
Got: 9954
Got: 9953
Got: 9952
Got: 9951
Got: 9950
Got: 9949
Got: 9948
Got: 9947
Got: 9946
Got: 9945
Got: 9944
Got: 9943
Got: 9942
Got: 9941
Got: 9940
Got: 9939
Got: 9938
Got: 9937
Got: 9936
Got: 9935
Got: 9934
Got: 9933
Got: 9932
Got: 9931
Got: 9930
Got: 9929
Got: 9928
Got: 9927
Got: 9926
Got: 9925
Got: 9924
Got: 9923
Got: 9922
Got: 9921
Got: 9920
Got: 9919
Got: 9918
Got: 9917
Got: 9916
Got: 9915
Got: 9914
Got: 9913
Got: 9912
Got: 9911
Got: 9910
Got: 9909
Got: 9908
Got: 9907
Got: 9906
Got: 9905
Got: 9904
Got: 9903
Got: 9902
Got: 9901

Got: 8568
Got: 8567
Got: 8566
Got: 8565
Got: 8564
Got: 8563
Got: 8562
Got: 8561
Got: 8560
Got: 8559
Got: 8558
Got: 8557
Got: 8556
Got: 8555
Got: 8554
Got: 8553
Got: 8552
Got: 8551
Got: 8550
Got: 8549
Got: 8548
Got: 8547
Got: 8546
Got: 8545
Got: 8544
Got: 8543
Got: 8542
Got: 8541
Got: 8540
Got: 8539
Got: 8538
Got: 8537
Got: 8536
Got: 8535
Got: 8534
Got: 8533
Got: 8532
Got: 8531
Got: 8530
Got: 8529
Got: 8528
Got: 8527
Got: 8526
Got: 8525
Got: 8524
Got: 8523
Got: 8522
Got: 8521
Got: 8520
Got: 8519
Got: 8518
Got: 8517
Got: 8516
Got: 8515
Got: 8514
Got: 8513
Got: 8512
Got: 8511
Got: 8510
Got: 8509
Got: 8508
Got: 8507
Got: 8506
Got: 8505
Got: 8504
Got: 8503
Got: 8502
Got: 8501
Got: 8500
Got: 8499
Got: 8498
Got: 8497
Got: 8496
Got: 8495
Got: 8494
Got: 8493
Got: 8492
Got: 8491
Got: 8490
Got: 8489
Got: 8488
Got: 8487
Got: 8486
Got: 8485
Got: 8484
Got: 8483
Got: 8482
Got: 8481
Got: 8480
Got: 8479
Got: 8478
Got: 8477
Got: 8476
Got: 8475
Got: 8474
Got: 8473
Got: 8472
Got: 8471
Got: 8470
Got: 8469


Got: 7068
Got: 7067
Got: 7066
Got: 7065
Got: 7064
Got: 7063
Got: 7062
Got: 7061
Got: 7060
Got: 7059
Got: 7058
Got: 7057
Got: 7056
Got: 7055
Got: 7054
Got: 7053
Got: 7052
Got: 7051
Got: 7050
Got: 7049
Got: 7048
Got: 7047
Got: 7046
Got: 7045
Got: 7044
Got: 7043
Got: 7042
Got: 7041
Got: 7040
Got: 7039
Got: 7038
Got: 7037
Got: 7036
Got: 7035
Got: 7034
Got: 7033
Got: 7032
Got: 7031
Got: 7030
Got: 7029
Got: 7028
Got: 7027
Got: 7026
Got: 7025
Got: 7024
Got: 7023
Got: 7022
Got: 7021
Got: 7020
Got: 7019
Got: 7018
Got: 7017
Got: 7016
Got: 7015
Got: 7014
Got: 7013
Got: 7012
Got: 7011
Got: 7010
Got: 7009
Got: 7008
Got: 7007
Got: 7006
Got: 7005
Got: 7004
Got: 7003
Got: 7002
Got: 7001
Got: 7000
Got: 6999
Got: 6998
Got: 6997
Got: 6996
Got: 6995
Got: 6994
Got: 6993
Got: 6992
Got: 6991
Got: 6990
Got: 6989
Got: 6988
Got: 6987
Got: 6986
Got: 6985
Got: 6984
Got: 6983
Got: 6982
Got: 6981
Got: 6980
Got: 6979
Got: 6978
Got: 6977
Got: 6976
Got: 6975
Got: 6974
Got: 6973
Got: 6972
Got: 6971
Got: 6970
Got: 6969


Got: 5318
Got: 5317
Got: 5316
Got: 5315
Got: 5314
Got: 5313
Got: 5312
Got: 5311
Got: 5310
Got: 5309
Got: 5308
Got: 5307
Got: 5306
Got: 5305
Got: 5304
Got: 5303
Got: 5302
Got: 5301
Got: 5300
Got: 5299
Got: 5298
Got: 5297
Got: 5296
Got: 5295
Got: 5294
Got: 5293
Got: 5292
Got: 5291
Got: 5290
Got: 5289
Got: 5288
Got: 5287
Got: 5286
Got: 5285
Got: 5284
Got: 5283
Got: 5282
Got: 5281
Got: 5280
Got: 5279
Got: 5278
Got: 5277
Got: 5276
Got: 5275
Got: 5274
Got: 5273
Got: 5272
Got: 5271
Got: 5270
Got: 5269
Got: 5268
Got: 5267
Got: 5266
Got: 5265
Got: 5264
Got: 5263
Got: 5262
Got: 5261
Got: 5260
Got: 5259
Got: 5258
Got: 5257
Got: 5256
Got: 5255
Got: 5254
Got: 5253
Got: 5252
Got: 5251
Got: 5250
Got: 5249
Got: 5248
Got: 5247
Got: 5246
Got: 5245
Got: 5244
Got: 5243
Got: 5242
Got: 5241
Got: 5240
Got: 5239
Got: 5238
Got: 5237
Got: 5236
Got: 5235
Got: 5234
Got: 5233
Got: 5232
Got: 5231
Got: 5230
Got: 5229
Got: 5228
Got: 5227
Got: 5226
Got: 5225
Got: 5224
Got: 5223
Got: 5222
Got: 5221
Got: 5220
Got: 5219


Got: 3568
Got: 3567
Got: 3566
Got: 3565
Got: 3564
Got: 3563
Got: 3562
Got: 3561
Got: 3560
Got: 3559
Got: 3558
Got: 3557
Got: 3556
Got: 3555
Got: 3554
Got: 3553
Got: 3552
Got: 3551
Got: 3550
Got: 3549
Got: 3548
Got: 3547
Got: 3546
Got: 3545
Got: 3544
Got: 3543
Got: 3542
Got: 3541
Got: 3540
Got: 3539
Got: 3538
Got: 3537
Got: 3536
Got: 3535
Got: 3534
Got: 3533
Got: 3532
Got: 3531
Got: 3530
Got: 3529
Got: 3528
Got: 3527
Got: 3526
Got: 3525
Got: 3524
Got: 3523
Got: 3522
Got: 3521
Got: 3520
Got: 3519
Got: 3518
Got: 3517
Got: 3516
Got: 3515
Got: 3514
Got: 3513
Got: 3512
Got: 3511
Got: 3510
Got: 3509
Got: 3508
Got: 3507
Got: 3506
Got: 3505
Got: 3504
Got: 3503
Got: 3502
Got: 3501
Got: 3500
Got: 3499
Got: 3498
Got: 3497
Got: 3496
Got: 3495
Got: 3494
Got: 3493
Got: 3492
Got: 3491
Got: 3490
Got: 3489
Got: 3488
Got: 3487
Got: 3486
Got: 3485
Got: 3484
Got: 3483
Got: 3482
Got: 3481
Got: 3480
Got: 3479
Got: 3478
Got: 3477
Got: 3476
Got: 3475
Got: 3474
Got: 3473
Got: 3472
Got: 3471
Got: 3470
Got: 3469


Got: 1819
Got: 1818
Got: 1817
Got: 1816
Got: 1815
Got: 1814
Got: 1813
Got: 1812
Got: 1811
Got: 1810
Got: 1809
Got: 1808
Got: 1807
Got: 1806
Got: 1805
Got: 1804
Got: 1803
Got: 1802
Got: 1801
Got: 1800
Got: 1799
Got: 1798
Got: 1797
Got: 1796
Got: 1795
Got: 1794
Got: 1793
Got: 1792
Got: 1791
Got: 1790
Got: 1789
Got: 1788
Got: 1787
Got: 1786
Got: 1785
Got: 1784
Got: 1783
Got: 1782
Got: 1781
Got: 1780
Got: 1779
Got: 1778
Got: 1777
Got: 1776
Got: 1775
Got: 1774
Got: 1773
Got: 1772
Got: 1771
Got: 1770
Got: 1769
Got: 1768
Got: 1767
Got: 1766
Got: 1765
Got: 1764
Got: 1763
Got: 1762
Got: 1761
Got: 1760
Got: 1759
Got: 1758
Got: 1757
Got: 1756
Got: 1755
Got: 1754
Got: 1753
Got: 1752
Got: 1751
Got: 1750
Got: 1749
Got: 1748
Got: 1747
Got: 1746
Got: 1745
Got: 1744
Got: 1743
Got: 1742
Got: 1741
Got: 1740
Got: 1739
Got: 1738
Got: 1737
Got: 1736
Got: 1735
Got: 1734
Got: 1733
Got: 1732
Got: 1731
Got: 1730
Got: 1729
Got: 1728
Got: 1727
Got: 1726
Got: 1725
Got: 1724
Got: 1723
Got: 1722
Got: 1721
Got: 1720


Got: 69
Got: 68
Got: 67
Got: 66
Got: 65
Got: 64
Got: 63
Got: 62
Got: 61
Got: 60
Got: 59
Got: 58
Got: 57
Got: 56
Got: 55
Got: 54
Got: 53
Got: 52
Got: 51
Got: 50
Got: 49
Got: 48
Got: 47
Got: 46
Got: 45
Got: 44
Got: 43
Got: 42
Got: 41
Got: 40
Got: 39
Got: 38
Got: 37
Got: 36
Got: 35
Got: 34
Got: 33
Got: 32
Got: 31
Got: 30
Got: 29
Got: 28
Got: 27
Got: 26
Got: 25
Got: 24
Got: 23
Got: 22
Got: 21
Got: 20
Got: 19
Got: 18
Got: 17
Got: 16
Got: 15
Got: 14
Got: 13
Got: 12
Got: 11
Got: 10
Got: 9
Got: 8
Got: 7
Got: 6
Got: 5
Got: 4
Got: 3
Got: 2
Got: 1


## 다중 스레드 큐 폴링

In [1]:
import queue
import socket
import os

class PollableQueue(queue.Queue):
    def __init__(self):
        super().__init__()
        # Create a pair of connected sockets        
        if os.name == 'posix':
            self._putsocket, self._getsocket = socket.socketpair()
        else:
            # Compatibility on non-POSIX systems
            server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            server.bind(('127.0.0.1', 0))
            server.listen(1)
            self._putsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self._putsocket.connect(server.getsockname())
            self._getsocket, _ = server.accept()
            server.close()

    def fileno(self):
        return self._getsocket.fileno()

    def put(self, item):
        super().put(item)
        self._putsocket.send(b'x')

    def get(self):
        self._getsocket.recv(1)
        return super().get()

# Example code that performs polling:

if __name__ == '__main__':
    import select
    import threading
    import time

    def consumer(queues):
        '''
        Consumer that reads data on multiple queues simultaneously
        '''
        while True:
            can_read, _, _ = select.select(queues,[],[])
            for r in can_read:
                item = r.get()
                print('Got:', item)

    q1 = PollableQueue()
    q2 = PollableQueue()
    q3 = PollableQueue()
    t = threading.Thread(target=consumer, args=([q1,q2,q3],))
    t.daemon = True
    t.start()

    # Feed data to the queues
    q1.put(1)
    q2.put(10)
    q3.put('hello')
    q2.put(15)

    # Give thread time to run
    time.sleep(1)

Got: 1
Got: 10
Got: hello
Got: 15
